this notebook is used to calculated the standard deviation of the convertion rate to select the delta used in the UCB Active

In [22]:
import numpy as np
import matplotlib.pyplot as plt


import Code.utils as utils
import Code.environment.settings as settings
from Code.environment.Environment import Environment
from Code.GreedyLearner import GreedyLearner
from Code.environment.Customer import Customer
from Code.UCBLearner import UCBLearner
from Code.TSLearner import TSLearner

%matplotlib inline

In [23]:
cust_json = "customer_classes.json"
today_customer = 100
var_cust = 0
p_lambda = 1.0
graph_json = "business_full_graph.json"

arms = settings.prices
environment = Environment(cust_json,today_customer,var_cust,p_lambda, graph_json, arms)
clairvoyant_arms, clairvoyant_reward, exp_c, rew_per_arm= environment.estimate_clairvoyant()
print("\n", clairvoyant_arms, clairvoyant_reward, exp_c, rew_per_arm.values())

loading: [==============================] 99.6%
 [0, 2, 0, 2, 0] 970.7239928 9.707239928 dict_values([752.6665272500001, 693.03838697, 664.4090954200001, 612.0842320700001, 755.492893625, 696.438626435, 668.08338141, 616.290364985, 766.677089, 707.8286694800001, 679.56990328, 628.00226588, 764.833268125, 706.2988147750001, 678.1905866500001, 626.902387525, 734.2235987500001, 674.9221025500002, 646.4497713000002, 594.40910305, 736.8712118750001, 678.1281130250001, 649.92291615, 598.3906682749999, 747.3171700000001, 688.7953492, 660.6947592, 609.3877612, 745.6002593750001, 687.3769441249999, 659.41957475, 608.3764403750001, 725.614559125, 666.3702632850001, 637.92176091, 586.000347635, 728.0888933125001, 669.3930736175001, 641.207364305, 589.7776442925, 737.9615665000001, 679.4994404400001, 651.4248554400002, 600.2207098400002, 736.3202315625001, 678.1458583875001, 650.2094273250001, 599.2578857625001, 678.2049264999999, 620.41278822, 592.6855933200002, 541.58091362, 680.7972857499999, 6

In [24]:
n_arms = 4
n_products = 5 

feature1 = 0
feature2 = 0

customer = Customer(feature1, feature2)

TIME_HORIZON = 300
NUM_ITERATION = 2

In [25]:
aggregate_customer = Customer(
    0,
    0,
    environment.get_aggregate_alphas(),
    environment.get_aggregate_num_prods_distribution(),
    environment.get_aggregate_click_graph()
    )


In [ ]:
rewards_iterations = []
conv_rate_hist = []
for i in range(NUM_ITERATION):
    ucb1_learner = UCBLearner(n_arms, n_products, aggregate_customer, graph_json, arms)
    print('Iteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ucb1_learner.select_superarm()
        report = environment.round(super_arm)
        ucb1_learner.update(super_arm.copy(), report)
        conv_rate_hist.append(report.get_conversion_rate())
    rewards = ucb1_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape

: 

In [26]:
rewards_iterations = []
conv_rate_hist = []
for i in range(NUM_ITERATION):
    ts_learner = TSLearner(n_arms, n_products, aggregate_customer, graph_json, arms)
    print('\nIteration #', i+1)
    for t in range(TIME_HORIZON):
        if t % 10 == 0:
            utils.progress_bar(t, TIME_HORIZON)
        super_arm = ts_learner.select_superarm()
        report = environment.round(super_arm)
        ts_learner.update(super_arm.copy(), report)
        conv_rate_hist.append(report.get_conversion_rate())
    rewards = ts_learner.history_rewards
    rewards_iterations.append(np.array(rewards))
rewards_iterations = np.array(rewards_iterations)
rewards_iterations.shape


Iteration # 1
loading: [============================= ] 96.7%
Iteration # 2
loading: [============================= ] 96.7%

(2, 300)

In [27]:
a = np.array(conv_rate_hist)
print(np.shape(a))

(600, 5)


In [28]:
delta = np.std(a, axis=0)

In [29]:
print(delta)

[0.23676673 0.09097285 0.14838332 0.19692154 0.11289689]
